In [119]:
import pandas as pd
from sqlalchemy import create_engine

from config import postgresLogin, postgresPwd
postgresLogin

'postgres'

In [30]:
nycDog_file = "./data/NYC_Dog_Licensing_Dataset.csv"
nycDog_df = pd.read_csv(nycDog_file)
nycDog_df.head()

,RowNumber,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year
0,1,PAIGE,F,2014,American Pit Bull Mix / Pit Bull Mix,NaN,10035,09/12/2014,09/12/2017,2016
1,2,YOGI,M,2010,Boxer,NaN,10465,09/12/2014,10/02/2017,2016
2,3,ALI,M,2014,Basenji,NaN,10013,09/12/2014,09/12/2019,2016
3,4,QUEEN,F,2013,Akita Crossbreed,NaN,10013,09/12/2014,09/12/2017,2016
4,5,LOLA,F,2009,Maltese,NaN,10028,09/12/2014,10/09/2017,2016


In [23]:
nycDog_df.sort_values("RowNumber", inplace = True)
nycDog_df.head(10)

,RowNumber,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year
112719,1,PAIGE,F,2014,American Pit Bull Mix / Pit Bull Mix,NaN,10035,09/12/2014,09/12/2017,2017
228356,1,ALI,M,2014,Basenji,NaN,10013,09/12/2014,09/12/2019,2018
228357,2,IAN,M,2006,Unknown,NaN,10013,09/12/2014,10/30/2019,2018
1,2,YOGI,M,2010,Boxer,NaN,10465,09/12/2014,10/02/2017,2016
112720,2,ALI,M,2014,Basenji,NaN,10013,09/12/2014,09/12/2019,2017
112721,3,QUEEN,F,2013,Akita Crossbreed,NaN,11368,09/12/2014,09/12/2017,2017
2,3,ALI,M,2014,Basenji,NaN,10013,09/12/2014,09/12/2019,2016
228358,3,CHEWBACCA,F,2012,Labrador Retriever Crossbreed,NaN,10013,09/12/2014,10/01/2019,2018
228359,4,LOLA,F,2006,Miniature Pinscher,NaN,10022,09/13/2014,10/03/2019,2018
3,4,QUEEN,F,2013,Akita Crossbreed,NaN,10013,09/12/2014,09/12/2017,2016


In [20]:
nycDog_df.shape
nycDog_df.columns


Index(['RowNumber', 'AnimalName', 'AnimalGender', 'AnimalBirthMonth',
       'BreedName', 'Borough', 'ZipCode', 'LicenseIssuedDate',
       'LicenseExpiredDate', 'Extract Year'],
      dtype='object')

In [21]:
#nycDog_df.drop_duplicates(subset=["RowNumber", "AnimalName", "BreadName"], keep='last' , inplace=True)

nycDog_df.drop_duplicates(subset=["RowNumber", "AnimalName", "BreedName"], keep='last' , inplace=True)

nycDog_df.shape

(345696, 10)

In [91]:
nycDog_df_cols = ['BreedName', 'ZipCode']
nycDog_df_transformed= nycDog_df[nycDog_df_cols].copy()
nycDog_df_transformed.shape
nycDog_df_transformed.head()

,BreedName,ZipCode
112719,American Pit Bull Mix / Pit Bull Mix,10035
228356,Basenji,10013
228357,Unknown,10013
1,Boxer,10465
112720,Basenji,10013


In [92]:
# What breeds are in the dataset?
breedNames = nycDog_df_transformed.BreedName.unique()
print(breedNames.shape)
breedNames.sort()
breedNames

(319,)


array(['Affenpinscher', 'Afghan Hound', 'Afghan Hound Crossbreed',
       'Airedale Terrier', 'Akita', 'Akita Crossbreed',
       'Alaskan Malamute', 'American Bully', 'American English Coonhound',
       'American Eskimo dog', 'American Foxhound',
       'American Hairless Terrier', 'American Leopard Hound',
       'American Pit Bull Mix / Pit Bull Mix',
       'American Pit Bull Terrier/Pit Bull',
       'American Staffordshire Terrier', 'American Water Spaniel',
       'Anatolian Shepherd Dog', 'Australian Cattle Dog',
       'Australian Cattledog', 'Australian Kelpie', 'Australian Shepherd',
       'Australian Silky Terrier', 'Australian Terrier', 'Azawakh',
       'Baladi', 'Barbet', 'Basenji', 'Basset Hound',
       'Basset Hound Crossbreed', 'Bassett Hound', 'Beagle',
       'Beagle Crossbreed', 'Bearded Collie', 'Beauceron',
       'Bedlington Terrier', 'Belgian Griffon', 'Belgian Laekenois',
       'Belgian Malinois', 'Belgian Sheepdog', 'Belgian Tervuren',
       'Berger Pica

In [95]:
## breed names to keep will come from the CatsaNDog Rating json dataset
## filter the NYC Dog data by breedNames we want to keep in the data

breedNamesFromRatings = ['Affenpinscher', 'Afghan Hound', 'Airedale Terrier', 'Akita',
       'Alaskan Klee Kai', 'Alaskan Malamute', 'American Bulldog',
       'American English Coonhound', 'American Eskimo Dog',
       'American Foxhound', 'American Pit Bull Terrier',
       'American Staffordshire Terrier', 'American Water Spaniel',
       'Anatolian Shepherd Dog', 'Appenzeller Sennenhunde',
       'Australian Cattle Dog', 'Australian Kelpie',
       'Australian Shepherd', 'Australian Terrier', 'Azawakh', 'Barbet',
       'Basenji', 'Basset Hound', 'Beagle', 'Bearded Collie',
       'Bedlington Terrier', 'Belgian Malinois', 'Belgian Sheepdog',
       'Belgian Tervuren', 'Berger Picard', 'Bernedoodle',
       'Bernese Mountain Dog', 'Bichon Frise', 'Black Mouth Cur',
       'Black Russian Terrier', 'Black and Tan Coonhound', 'Bloodhound',
       'Blue Lacy', 'Bluetick Coonhound', 'Boerboel', 'Bolognese',
       'Border Collie', 'Border Terrier', 'Borzoi', 'Boston Terrier',
       'Bouvier des Flandres', 'Boxer', 'Boykin Spaniel',
       'Bracco Italiano', 'Briard', 'Brittany', 'Brussels Griffon',
       'Bull Terrier', 'Bulldog', 'Bullmastiff', 'Cairn Terrier',
       'Canaan Dog', 'Cane Corso', 'Cardigan Welsh Corgi',
       'Catahoula Leopard Dog', 'Caucasian Shepherd Dog',
       'Cavalier King Charles Spaniel', 'Cesky Terrier',
       'Chesapeake Bay Retriever', 'Chihuahua', 'Chinese Crested',
       'Chinese Shar-Pei', 'Chinook', 'Chow Chow', 'Clumber Spaniel',
       'Cockapoo', 'Cocker Spaniel', 'Collie', 'Coton de Tulear',
       'Curly-Coated Retriever', 'Dachshund', 'Dalmatian',
       'Dandie Dinmont Terrier', 'Doberman Pinscher', 'Dogo Argentino',
       'Dogue de Bordeaux', 'Dutch Shepherd', 'English Cocker Spaniel',
       'English Foxhound', 'English Setter', 'English Springer Spaniel',
       'English Toy Spaniel', 'Entlebucher Mountain Dog', 'Field Spaniel',
       'Finnish Lapphund', 'Finnish Spitz', 'Flat-Coated Retriever',
       'Fox Terrier', 'French Bulldog', 'German Pinscher',
       'German Shepherd Dog', 'German Shorthaired Pointer',
       'German Wirehaired Pointer', 'Giant Schnauzer',
       'Glen of Imaal Terrier', 'Goldador', 'Golden Retriever',
       'Goldendoodle', 'Gordon Setter', 'Great Dane', 'Great Pyrenees',
       'Greater Swiss Mountain Dog', 'Greyhound', 'Harrier', 'Havanese',
       'Ibizan Hound', 'Icelandic Sheepdog', 'Irish Red and White Setter',
       'Irish Setter', 'Irish Terrier', 'Irish Water Spaniel',
       'Irish Wolfhound', 'Italian Greyhound', 'Jack Russell Terrier',
       'Japanese Chin', 'Japanese Spitz', 'Karelian Bear Dog', 'Keeshond',
       'Kerry Blue Terrier', 'Komondor', 'Kooikerhondje', 'Kuvasz',
       'Labradoodle', 'Labrador Retriever', 'Lagotto Romagnolo',
       'Lakeland Terrier', 'Lancashire Heeler', 'Leonberger',
       'Lhasa Apso', 'Lowchen', 'Maltese', 'Maltese Shih Tzu', 'Maltipoo',
       'Manchester Terrier', 'Mastiff', 'Miniature Pinscher',
       'Miniature Schnauzer', 'Mudi', 'Mutt', 'Neapolitan Mastiff',
       'Newfoundland', 'Norfolk Terrier', 'Norwegian Buhund',
       'Norwegian Elkhound', 'Norwegian Lundehund', 'Norwich Terrier',
       'Nova Scotia Duck Tolling Retriever', 'Old English Sheepdog',
       'Otterhound', 'Papillon', 'Peekapoo', 'Pekingese',
       'Pembroke Welsh Corgi', 'Petit Basset Griffon Vendeen',
       'Pharaoh Hound', 'Plott', 'Pocket Beagle', 'Pointer',
       'Polish Lowland Sheepdog', 'Pomeranian', 'Pomsky', 'Poodle',
       'Portuguese Water Dog', 'Pug', 'Puggle', 'Puli',
       'Pyrenean Shepherd', 'Rat Terrier', 'Redbone Coonhound',
       'Rhodesian Ridgeback', 'Rottweiler', 'Saint Bernard', 'Saluki',
       'Samoyed', 'Schipperke', 'Schnoodle', 'Scottish Deerhound',
       'Scottish Terrier', 'Sealyham Terrier', 'Shetland Sheepdog',
       'Shiba Inu', 'Shih Tzu', 'Siberian Husky', 'Silken Windhound',
       'Silky Terrier', 'Skye Terrier', 'Sloughi',
       'Small Munsterlander Pointer', 'Soft Coated Wheaten Terrier',
       'Stabyhoun', 'Staffordshire Bull Terrier', 'Standard Schnauzer',
       'Sussex Spaniel', 'Swedish Vallhund', 'Tibetan Mastiff',
       'Tibetan Spaniel', 'Tibetan Terrier', 'Toy Fox Terrier',
       'Treeing Tennessee Brindle', 'Treeing Walker Coonhound', 'Vizsla',
       'Weimaraner', 'Welsh Springer Spaniel', 'Welsh Terrier',
       'West Highland White Terrier', 'Whippet',
       'Wirehaired Pointing Griffon', 'Yorkipoo', 'Yorkshire Terrier']

nycDog_df_transformed = nycDog_df_transformed[nycDog_df_transformed.BreedName.isin(breedNamesFromRatings)]



In [114]:
#  Most poppular breeds in NYC. 
# Can choose these to filter the data set even more
breedcounts_df = nycDog_df_transformed.groupby(['BreedName']).count()
breedcounts_df = breedcounts_df.sort_values(by=['ZipCode'], ascending=False)
breedcounts_df.rename(columns={'ZipCode' : 'Amount'}, inplace=True)

breedcounts_df

,Amount
BreedName,
Yorkshire Terrier,21921
Shih Tzu,19628
Chihuahua,15645
Maltese,11391
Labrador Retriever,11326
...,...
Bracco Italiano,3
Finnish Lapphund,2
Azawakh,1


In [115]:
breedcounts_df.shape

(191, 1)

# Create database connection

In [122]:
connection_string = f"{postgresLogin}:{postgresPwd}@localhost:5432/nyc_dogs_db"
engine = create_engine(f'postgresql://{connection_string}')

# Confirm tables

In [123]:
engine.table_names()

['nyc_dog', 'dog_rating']

# Load DataFrames into database

In [118]:
breedcounts_df.to_sql(name='nyc_dog', con=engine, if_exists='append', index=True)
